# Improving LLM Metalinguistic Knowledge Incorporating Fundéu's Recommendations through RAG


Reyes Gago Sosa and Amaia Murillo Lekuona

Currently, Large Language Models (LLMs) have shown remarkable proficiency and success in various language-related tasks. However, they still face limitations, particularly in domain-specific applications, where they often produce 'hallucinations' when handling queries that require up-to-date information or fall outside their training data. To address these challenges, Retrieval-Augmented Generation (RAG) enhances LLMs by retrieving relevant document chunks from external knowledge sources ([Gao et al., 2023](https://arxiv.org/pdf/2312.10997)).

This project aims to improve LLM metalinguistic knowledge incorporating FundéuRAE's recommendations for correct use of Spanish language through RAG. To this end, in the first part of this Notebook we create a dataset based on all the linguistic recommendations from [FundéuRAE](https://www.fundeu.es/) based on web scrapping. Following this, 10 questions are selected, each with answers found within the dataset. After that, we implement an experimental question answering system using information retrieval with the top 3 most relevant documents. Subsequently, a comparison is made between the responses to the questions by the [Mistral7B-instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) model before and after the implementation of RAG, in order to qualitatively assess its effect. Finally, the creation of a chatbot utilizing this information is proposed.

The linguistic queries that will be asked to the model will be:

1. ¿Se puede considerar 'gepetear' un neologismo?

2. ¿Cuáles son los equivalentes de 'password' en español?

3. ¿Cómo se escribe 'reduflacción' o 'reduflación'?

4. ¿Cómo se acentúa 'adecua' o 'adecúa'?

5. ¿Cuál es la diferencia entre 'sino' y 'si no'?

6. ¿Cuál es la forma adecuada para abreviar 'Reino Unido'?

7. ¿La palabra 'solo' se escribe con tilde cuando es adverbio?

8. ¿Cómo se escriben los títulos de las obras de creación?

9. ¿Se puede escribir coma entre sujeto y verbo de una oración?

10. ¿Cuál es la forma recomendada para referirse a los habitantes de Ucrania?






## Create the dataset

### Fundéu
The Fundación del Español Urgente (FundéuRAE) is a non-profit institution whose main objective is to promote the proper use of Spanish in the media. Promoted by the Royal Spanish Academy (RAE) and the EFE Agency, it was created in 2005 as a result of an agreement between EFE and the BBVA bank, with the advice of the Royal Spanish Academy.

It publishes a daily recommendation on the correct use of Spanish, based on the doubts detected in the main Spanish-language media. These tips are published on its website (under Creative Commons licence) and distributed through social networks.

The FundéuRAE website is structured into two key sections: 'Recommendations' and 'Inquiries'. Our project will specifically focus on the 'Recommendations' section, which provides daily expert advice on the proper use of Spanish. We intend to compile all recommendations available by FundéuRAE up to the current date. Therefore, the dataset will include the latest available recommendation, which is dated 3 May 2024.

The recommendations are divided into 5 categories:

- Fonética
- Gramática
- Léxico
- Ortografía
- Otros



### Web Scraping
To create our dataset, we will automate the extraction of all FundéuRAE's recommendations through web scraping. To achieve this, we are using Beautiful Soup library to parse the website’s HTML content efficiently.

In [ ]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

To understand the structure of the FundéuRAE recommendations, we will first obtain the HTML code of a single recommendation so as to examine the structure of the webpage and to identify the HTML tags that contain the recommendation text.

In [ ]:
# URL of the Fundéu recommendation page
url = 'https://www.fundeu.es/recomendacion/hora-local-225/'

# Perform the HTTP request to the URL
response = requests.get(url)

# Extract the HTML content from the response
html = response.text

In [ ]:
html

'<!DOCTYPE HTML><html lang="es"><head><link media="all" href="https://www.fundeu.es/wp-content/cache/autoptimize/css/autoptimize_c4e5554afebbd0b5f049c1ff8885a804.css" rel="stylesheet"><link media="print" href="https://www.fundeu.es/wp-content/cache/autoptimize/css/autoptimize_7d176543f250ec625198de9353939a7b.css" rel="stylesheet"><link media="screen" href="https://www.fundeu.es/wp-content/cache/autoptimize/css/autoptimize_616fe659876e3c5516576de49483afd5.css" rel="stylesheet"><title>«horas locales», uso inadecuado | FundéuRAE</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, height=device-height, user-scalable=no, maximum-scale=.8, target-densitydpi=device-dpi"><link rel="profile" href="https://gmpg.org/xfn/11" /><link rel="pingback" href="https://www.fundeu.es/xmlrpc.php" /><meta property="og:title" content="«horas locales», uso inadecuado" /><meta property="og:type" content="article" 

In [ ]:
import requests
from bs4 import BeautifulSoup

def retrieve_text(url):
    """
    This function takes a URL, makes an HTTP request, and extracts and cleans the text from the page.
    Args:
    url (str): URL of the webpage from which the text will be extracted.

    Returns:
    str: Clean text extracted from the page, or an error message if the content is not found.
    """
    # Perform the HTTP request to the URL
    response = requests.get(url)

    # Check that the request was successful
    if response.status_code != 200:
        return "Error in obtaining the webpage: HTTP Status {}".format(response.status_code)

    # Use BeautifulSoup to parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the main content div
    content_div = soup.find('div', class_='art-content mod-inner')

    # Initialize a variable to store the complete text
    full_text = ""

    # Check if the div is not empty
    if content_div:
        # Find all paragraphs within the div
        paragraphs = content_div.find_all('p')

        # Extract the text from each paragraph and concatenate until the desired end is reached
        for paragraph in paragraphs:
            # Stop accumulation upon detecting the start of the undesired section
            if paragraph.find_previous_sibling('h4') and "Ver también" in paragraph.find_previous_sibling('h4').get_text():
                break
            # Remove new lines and concatenate directly
            text = paragraph.get_text(" ", strip=True)
            full_text += text + " "

        # Correct punctuation in the complete text
        full_text = full_text.strip()
        corrections = [
            (" ,", ","), (" .", "."), (" ;", ";"), (" :", ":"),
            (" !", "!"), (" ?", "?"), (" %", "%"), ("( ", "("),
            (" )", ")"), (" / ", "/")
        ]
        for old, new in corrections:
            full_text = full_text.replace(old, new)

        # Return the clean, final text
        return full_text
    else:
        return "No content container found."

# Retrieve the text from the latest recommendation of the FundéuRAE
url = 'https://www.fundeu.es/recomendacion/hora-local-225/'
print(retrieve_text(url))

Cuando en la mención de las horas se emplea la expresión hora local, no es adecuado el uso del plural horas locales. Sin embargo, a veces se encuentran en los medios de comunicación frases como «Este viernes, alrededor de las 10:23, horas locales, se registró un terremoto en Nueva York», «Los pasajeros no lograron regresar a tiempo para la última llamada a bordo, que estaba programada para las 15:00, horas locales» o «La exgobernadora pronunció un discurso en Charleston a las 10:00, horas locales». El uso del plural horas locales en estos casos es impropio porque lo que se quiere explicar es que la hora dada es la local, es decir, la hora concreta en el lugar donde ocurrió el suceso. Por tanto, en los ejemplos iniciales, lo adecuado habría sido escribir «Este viernes, alrededor de las 10:23, hora local, se registró un terremoto en Nueva York», «Los pasajeros no lograron regresar a tiempo para la última llamada a bordo, que estaba programada para las 15:00, hora local» y «La exgobernado

In [ ]:
# Check if the function works correctly with another URL
print(retrieve_text('https://www.fundeu.es/recomendacion/password-alternativas-2/'))

Contraseña y clave/código de acceso son algunos de los equivalentes del extranjerismo inglés password. Sin embargo, es frecuente ver frases en la prensa como «Antes de restaurar el password, mejor utiliza estos sistemas para saber cuál es», «Es recomendable cambiar tu password con regularidad para evitar que alguien la descifre» o «Para conseguir un password relativamente seguro hay que alcanzar los diez caracteres». Con el sentido de ‘secuencia de caracteres que permite el acceso a un sistema’, con el que se emplea el anglicismo password, existen opciones en español como contraseña, clave/código de acceso, código de seguridad o clave personal, tal como señala el Diccionario panhispánico de dudas. Por tanto, es innecesario el uso del extranjerismo y se recomienda optar por alguna de las alternativas anteriores. Así, en las oraciones iniciales, lo idóneo habría sido escribir «Antes de restaurar la contraseña, mejor utiliza estos sistemas para saber cuál es», «Es recomendable cambiar tu 

The website includes a section called ['Categorías'](https://www.fundeu.es/categorias/), which organizes recommendations into different types of linguistic queries. From this section, we can access all the queries and recommendations that have been published up to the present.

The extraction process will be conducted in two main stages:
* **Extracting Category URLs:** Initially, we will access the category section to obtain URLs for each specific category.
* **Extracting Recommendations by Category:** Once we have the category URLs, we will automate the extraction of all recommendations within each category.

Just as we previously extracted the text from an individual recommendation, we now retrieve the HTML from the 'Categorías' section to analyze the HTML structure to identify the specific tags that enclose the URLs for each category of linguistic queries.

In [ ]:
url = 'https://www.fundeu.es/categorias/'
response = requests.get(url)
html = response.text

In [ ]:
html

'<!DOCTYPE HTML><html lang="es"><head><link media="all" href="https://www.fundeu.es/wp-content/cache/autoptimize/css/autoptimize_c4e5554afebbd0b5f049c1ff8885a804.css" rel="stylesheet"><link media="print" href="https://www.fundeu.es/wp-content/cache/autoptimize/css/autoptimize_7d176543f250ec625198de9353939a7b.css" rel="stylesheet"><link media="screen" href="https://www.fundeu.es/wp-content/cache/autoptimize/css/autoptimize_616fe659876e3c5516576de49483afd5.css" rel="stylesheet"><title>Categorías | FundéuRAE</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta charset="UTF-8" /><meta name="viewport" content="width=device-width, height=device-height, user-scalable=no, maximum-scale=.8, target-densitydpi=device-dpi"><link rel="profile" href="https://gmpg.org/xfn/11" /><link rel="pingback" href="https://www.fundeu.es/xmlrpc.php" /><meta property="og:title" content="FundéuRAE" /><meta property="og:type" content="blog" /><meta property="og:url" content="https://www

After analyzing the HTML structure of the 'Categorías' section, the next step is to create a function that can extract all URLs for each type of linguistic query. It should be noted that we have observed that all URLs linked to specific types of linguistic queries begin with 'https://www.fundeu.es/dudas/tipo-de-duda/'.

In [ ]:
def extract_categories():
    """
    Extracts URLs for different types of linguistic inquiries from the FundéuRAE 'Categorías' section.

    Returns:
    list: A list of the URLs, each representing a different linguistic category.
    """

    url = 'https://www.fundeu.es/categorias/'
    categories = ['fonetica', 'gramatica', 'lexico', 'ortografia', 'visualizacion']
    specific_urls = []
    response = requests.get(url)

    # Check that the request was successful
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Filter links based on the specific starting pattern and category keywords
        links = soup.find_all('a', href=True)
        for link in links:
            href = link['href']
            # Check if any of the category keywords is in the href and if it starts with 'https://www.fundeu.es/dudas/tipo-de-duda/'
            if any(category in href for category in categories) and href.startswith('https://www.fundeu.es/dudas/tipo-de-duda/'):
                specific_urls.append(href)
    else:
        print("Failed to fetch the page:", response.status_code)

    return specific_urls

# Extract the URLs of the categories
urls = extract_categories()
print("Found URLs:")
for url in urls:
    print(url)

Found URLs:
https://www.fundeu.es/dudas/tipo-de-duda/fonetica/
https://www.fundeu.es/dudas/tipo-de-duda/gramatica-11/
https://www.fundeu.es/dudas/tipo-de-duda/lexico-2/
https://www.fundeu.es/dudas/tipo-de-duda/antroponimos-lexico/
https://www.fundeu.es/dudas/tipo-de-duda/toponimos-y-gentilicios-lexico-2/
https://www.fundeu.es/dudas/tipo-de-duda/ortografia-2/
https://www.fundeu.es/dudas/tipo-de-duda/visualizacion/


Once we have defined the function to extract URLs from the 'Categorías' section, our next objective is to create another function designed to retrieve all recommendation URLs from each category. We have observed that each of these recommendation URLs starts with 'https://www.fundeu.es/recomendacion/'.

In [ ]:
def extract_urls_from_category(base_url):
    """
    Extract all recommendation URLs from all pages of a specific category of linguistic doubts.

    Args:
    base_url (str): Base URL of the category of doubts.

    Returns:
    list: A list of all recommendation URLs found.
    """
    all_urls = []
    current_page = 1
    while True:
        # Construct the URL for the current page
        page_url = f"{base_url}/page/{current_page}/" if current_page > 1 else base_url
        response = requests.get(page_url)
        if response.status_code != 200:
            break  # Stop if the page is not accessible

        soup = BeautifulSoup(response.text, 'html.parser')
        # Find all links that start with the desired prefix
        links = soup.find_all('a', href=True)
        new_found = False
        for link in links:
            href = link['href']
            if href.startswith('https://www.fundeu.es/recomendacion/'):
                all_urls.append(href)
                new_found = True

        if not new_found:  # Break the loop if no new URLs are found on this page
            break

        current_page += 1  # Move to the next page

    return all_urls

# Extract all the recommendation URLs from a specific category
category_url = 'https://www.fundeu.es/dudas/tipo-de-duda/fonetica/'
recommendation_urls = extract_urls_from_category(category_url)
for url in recommendation_urls:
    print(url)

https://www.fundeu.es/recomendacion/ales-bialiatski-pronunciacion/
https://www.fundeu.es/recomendacion/liz-truss-pronunciacion/
https://www.fundeu.es/recomendacion/pergenar-no-perguenar/
https://www.fundeu.es/recomendacion/nicola-sturgeon/
https://www.fundeu.es/recomendacion/abu-ibrahim-al-qurashi-pronunciacion/
https://www.fundeu.es/recomendacion/abdulrazak-gurnah-pronunciacion/
https://www.fundeu.es/recomendacion/annalena-baerbock-pronunciacion/
https://www.fundeu.es/recomendacion/mark-rutte-pronunciacion/
https://www.fundeu.es/recomendacion/sputnik-v-lectura/
https://www.fundeu.es/recomendacion/ngozi-okonjo-iweala/
https://www.fundeu.es/recomendacion/min-aung-hlaing-pronunciacion/
https://www.fundeu.es/recomendacion/aung-san-suu-kyi-pronunciacion/
https://www.fundeu.es/recomendacion/amy-coney-barret-pronunciacion/
https://www.fundeu.es/recomendacion/yoshihide-suga-pronunciacion/
https://www.fundeu.es/recomendacion/alfred-schreuder-pronunciacion/
https://www.fundeu.es/recomendacion/o

In [ ]:
# Check if the number of URLs retrieved matches the number of recommendations listed in this section on the website.
print(f"Total URLs found: {len(recommendation_urls)}")

Total URLs found: 60


The subsequent step consists in extracting all the recommendation URLs. To achieve this, we create a function that employs the functions we have built above. First we obtain the category URLs and then we extract from each category URL all the recommendations.

In [ ]:
def extract_all_recommendations():
    category_urls = extract_categories()
    all_recommendations = []
    for category_url in category_urls:
        category_recommendations = extract_urls_from_category(category_url)
        all_recommendations.extend(category_recommendations)
    return all_recommendations

recommendation_urls = extract_all_recommendations()
for url in recommendation_urls:
    print(url)

Se han truncado las últimas 5000 líneas del flujo de salida.
https://www.fundeu.es/recomendacion/desemboca-en-desemboca-a/
https://www.fundeu.es/recomendacion/premios-goya/
https://www.fundeu.es/recomendacion/guino-referencia-o-cameo-alternativas-a-easter-egg-huevo-de-pascua-en-cine/
https://www.fundeu.es/recomendacion/revertir-revirtio-no-revertio/
https://www.fundeu.es/recomendacion/llevarotraer-aparejado-uso-apropiado-1438/
https://www.fundeu.es/recomendacion/difamar-y-acusar-no-son-equivalente/
https://www.fundeu.es/recomendacion/bendecidobendito/
https://www.fundeu.es/recomendacion/gabapentina-mejor-que-gabapentin-y-gabapentin/
https://www.fundeu.es/recomendacion/no-hay-manera-de-que-no-no-hay-manera-que/
https://www.fundeu.es/recomendacion/rali-dakar-claves-para-una-buena-redaccion/
https://www.fundeu.es/recomendacion/plebiscito-en-chile-claves-de-redaccion/
https://www.fundeu.es/recomendacion/azotar/
https://www.fundeu.es/recomendacion/desconfiar-de-que-no-desconfiar-que/
https:

Now that we have all the recommendation URLs, we use the `retrieve_text()` function in order to obtain the text from each URL, and we save the recommendations into a TXT file.

In [ ]:
def save_texts_to_file(urls, filename):
    """
    Takes a list of URLs, retrieves text from each, and saves the text to a specified file with each text entry separated by a newline.

    Args:
    urls (list): List of webpage URLs from which to extract text.
    filename (str): The name of the file where the texts will be saved, each entry separated by newlines.
    """
    with open(filename, 'w', encoding='utf-8') as file:
        for url in urls:
            try:
                text = retrieve_text(url)  # Retrieve text from the URL
                if not text.startswith("Error") and not text.startswith("No content"):
                    file.write(text + "\n\n")  # Write the text followed by two newlines
                else:
                    print(f"Failed to retrieve or no content found at: {url}")
            except Exception as e:
                print(f"An error occurred while processing {url}: {str(e)}")

In [ ]:
save_texts_to_file(recommendation_urls, 'recommendations.txt')

Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/previo-a-no-es-siempre-lo-mismo-que-antes-de-1254/
Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/fobo-fobico/
Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/blooper-alternativas-en-espanol/
Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/riesgoso/
Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/falacia/
Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/talgo/
Failed to retrieve or no content found at: https://www.fundeu.es/recomendacion/talgo/


The error messages indicate that some of the retrieved URLs do not correspond to valid recommendations. They are Fundéu linguistic queries instead of recommendations. The only exception is the URL 'https://www.fundeu.es/recomendacion/blooper-alternativas-en-espanol/', which is a valid recommendation but was not accepted in the process. We will manually correct the 'recommendations.txt' file to include this recommendation and then upload the updated file to our Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def count_recommendations(filename):
    count = 0
    with open(filename, 'r', encoding='utf-8') as file:
        content = file.read()
        entries = list(filter(bool, content.split("\n\n")))
        count = len(entries)

    return count

file_path = '/content/drive/My Drive/Colab Notebooks/nlp-app-II/Project/recommendations.txt'
texts_count = count_recommendations(file_path)
print(f"Total of recommendations: {texts_count}")


Total of recommendations: 5065


This dataset comprises 5,065 recommendations of the FundéuRAE, which cover a wide range of linguistic domains, including phonetics, grammar, lexis, orthography and orthotypography.

## Information Retrieval

In this section, we aim to develop a document retrieval system that identifies and extracts the most pertinent documents from a collection of documents, specifically our dataset, in response to a given query. This system will be used for both the Question Answering model and for the implementation of Retrieval Augmented Generation in order to improve the response of a model based on the recommendations of FundéuRAE, which will be explained in more detail in the RAG section.

In [ ]:
# Install the required libraries
!pip install Whoosh
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 9.9 MB/s eta 0:00:00


In [ ]:
# Import the required libraries
import codecs # base classes for standard Python codecs, like text encodings (UTF-8,...)
from IPython.core.display import display, HTML # object displaying in different formats
from whoosh.index import * # whoosh: full-text indexing and searching
from whoosh.fields import *
from whoosh import qparser
import glob
import random
import torch

Now, we create a function that reads de TXT file containing the recommendations and splits them into individual entries. Then, we count the total number of recommendations to ensure consistency with the expected number (5,065). Finally, we display three random recommendations to verify that the function is working correctly.

In [ ]:
def read_recommendations(path):
    with open(path, 'r') as f:
        content = f.read()
    recommendations = content.split('\n\n')
    count = len(recommendations)
    return recommendations, count

In [ ]:
path='/content/drive/My Drive/Colab Notebooks/nlp-app-II/Project/recommendations.txt'
recommendations, count = read_recommendations(path)
print("Number of linguistic recommendations:", count)
print()
print("3 random recommendations:")
print()
print(recommendations[random.randrange(count)])
print(recommendations[random.randrange(count)])
print(recommendations[random.randrange(count)])

Number of linguistic recommendations: 5065

3 random recommendations:

A continuación se ofrecen algunas claves para una correcta redacción de las informaciones relacionadas con los terremotos. Tanto sismo como seísmo son formas apropiadas para nombrar a un terremoto o sacudida de la tierra. La primera es más común en Hispanoamérica y la segunda en España. No significan lo mismo: mientras que el hipocentro, también llamado foco sísmico, es el lugar en el interior de la corteza terrestre donde tiene origen un sismo, el epicentro es el punto en la superficie terrestre —⁠aunque puede estar sumergido⁠— donde el terremoto es más intenso. Además, puesto que no son hechos o eventos, sino lugares o puntos de la corteza terrestre, lo apropiado es decir que se localizan, no que ocurren o se producen. La forma tsunami es la recomendada para aludir a ese tipo de ola gigantesca, si bien también es admisible la simplificada sunami. La escala de Richter mide la magnitud de un movimiento sísmico, la e

The Information Retrieval (IR) system we are developing has two principal modules: the indexing system and the query system. The indexing system is responsible for building the primary data structure, namely the index, so as to facilitate the document retrieval. As for the query system, it serves as the interactive interface of the IR system, where users enter their queries and the system uses the index to retrieve relevant documents. For building these systems, we will utilize the Whoosh library.


To create an index, it is necessary to define its schema, which specifies the fields that will be included in the index. These might include the document's ID, path, title and text content. We will set the type for the title and text field as 'TEXT' to allow searchability. Besides, we will apply a Stemming Analyzer to these text fields. This analyzer tokenizes the text, converts them to lowercase, removes stopwords and applies stemming to reduce words to their base forms.



In [ ]:
# Schema definition:
# - id: type ID, unique, stored; doc id in order given the passages file
# - text: type TEXT processed by StemmingAnalyzer; not stored; content of the passage
schema = Schema(id = ID(stored=True,unique=True),
                text = TEXT(analyzer=analysis.StemmingAnalyzer())
               )

After defining the schema, we will create an index.

In [ ]:
# Create an index
if not os.path.exists("index"):
    os.mkdir("index")

ix = create_in("index", schema)
writer = ix.writer() #run once! or restart runtime

Subsequently, we will add documents to the index. In other words, we will organize the recommendations provided by FundéuRAE adding each one to a searchable index. As we process each recommendation, it is cataloged with a unique identifier to ensure that each entry is distinct and retrievable.

In [ ]:
# Add papers to the index, iterating through each row in the metadata dataframe
for ind,recommendation_text in enumerate(recommendations):
    writer.add_document(id=str(ind),text=recommendation_text)

Finally, we will save the added documents to the index for future retrieval.

In [ ]:
# Save the added documents
writer.commit()
print("Index successfully created")

# Sanity check
print("Number of documents in the index: ", ix.doc_count())

Index successfully created
Number of documents in the index:  5065


Once we have added the recommendation to the index, we will define a function designed to take a question and a maximum number of documents as input. This function will use these inputs to perform a search query and retrieve relevant documents that were previously indexed. The algorith used for scoring will be the default BM25 and we will also set up the query parser, specifying 'text' as the default field for our search. The function will then execute the query an retrieves the most pertinents recommendations from the index, limited to a maximum number of documents.

In [ ]:
# Input: Question and maximum number of documents to retrieve
def retrieve_documents(query, topk):
    scores=[]
    text=[]
    # Open the searcher for reading the index. The default BM25 algorithm will be used for scoring
    with ix.searcher() as searcher:
        searcher = ix.searcher()

        # Define the query parser ('text' will be the default field to search), and set the input query
        q = qparser.QueryParser("text", ix.schema, group=qparser.OrGroup).parse(query)

        # Search using the query q, and get the topk documents, sorted with the highest-scoring documents first
        results = searcher.search(q, limit=topk)
        # results is a list of dictionaries where each dictionary is the stored fields of the document

    # Iterate over the retrieved documents
    for hit in results:
        scores.append(hit.score)
        text.append(recommendations[int(hit['id'])])
    return text, scores

Now, we will test the function by retrieving the 3 most relevant recommendations and their corresponding scores given the following query: '¿Cuál es el equivalente a password?'.

In [ ]:
retrieve_documents("¿Cuál es el equivalente a password?", 3)

(['Contraseña y clave/código de acceso son algunos de los equivalentes del extranjerismo inglés password. Sin embargo, es frecuente ver frases en la prensa como «Antes de restaurar el password, mejor utiliza estos sistemas para saber cuál es», «Es recomendable cambiar tu password con regularidad para evitar que alguien la descifre» o «Para conseguir un password relativamente seguro hay que alcanzar los diez caracteres». Con el sentido de ‘secuencia de caracteres que permite el acceso a un sistema’, con el que se emplea el anglicismo password, existen opciones en español como contraseña, clave/código de acceso, código de seguridad o clave personal, tal como señala el Diccionario panhispánico de dudas. Por tanto, es innecesario el uso del extranjerismo y se recomienda optar por alguna de las alternativas anteriores. Así, en las oraciones iniciales, lo idóneo habría sido escribir «Antes de restaurar la contraseña, mejor utiliza estos sistemas para saber cuál es», «Es recomendable cambiar 

## Question Answering

In this part, we want to develop a Question Answering (QA) system using the IR system we have just created. This exercise is designed only to observe how the system retrieves the most pertinent recommendations based on queries. However, **it is important to clarify that the results from this QA will not be included of the final evaluation.**

Given that our dataset is in Spanish, we initially attempted to use a Spanish-language QA model, '[PlanTL-GOB-ES/roberta-large-bne-sqac](https://huggingface.co/PlanTL-GOB-ES/roberta-large-bne-sqac)' from Hugging Face. However, the results were not satisfactory. Consequently, we switched to the '[bert-large-uncased-whole-word-masking-finetuned-squad](https://huggingface.co/google-bert/bert-large-uncased-whole-word-masking-finetuned-squad)' model, which is trained in English and shows better perfomance.



In [ ]:
from transformers import pipeline
from transformers import BertForQuestionAnswering, BertTokenizer

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The following two chunks of code are taken from [@juletx](https://github.com/juletx/nlp-applications-2/blob/master/assignments/assignment2.open_domain_Q.A.ipynb)'s GitHub repository. The first function, `answer_question()` consists of processing a question and the answer text, obtained through IR, in order to identify and extract the specifict text segment that answers the question.

In [ ]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    # Truncate if necessary to fit the model's maximum input size
    input_ids = tokenizer.encode(question, answer_text, max_length=512, truncation=True)

    # Report how long the input sequence is.
    # print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id)

    # The number of segment A tokens includes the [SEP] token itself.
    num_seg_a = sep_index + 1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b

    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example through the model.
    outputs = model(torch.tensor([input_ids]), # The tokens representing our input text.
                    token_type_ids=torch.tensor([segment_ids]), # The segment IDs to differentiate question from answer_text
                    return_dict=True)

    start_scores = outputs.start_logits[0][(num_seg_a):len(input_ids)-1]
    end_scores = outputs.end_logits[0][(num_seg_a):len(input_ids)-1]

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores) + num_seg_a
    answer_end = torch.argmax(end_scores) + num_seg_a

    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Start with the first token.
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]

    # Calculate the final score by adding the maximum start and end scores.
    score = (start_scores.max() + end_scores.max()).item()

    return answer, score

The second function, `best_answers()` processes a query and uses the previously defined `answer_question()` function to extract answers from the top retrieved documents. I evaluates each document, scores the answers based on their relevance and compiles the results into a dataframe sorted in descending order.

In [ ]:
import pandas as pd

def best_answers(query, count):
    pd.set_option('display.max_colwidth', None)
    answers = []
    answer_scores = []
    docs, scores = retrieve_documents(query, count)
    for i, doc in enumerate(docs):
        answer, score = answer_question(query, doc)
        answers.append(answer)
        answer_scores.append(score)
    df = pd.DataFrame({"Document": docs, "Document Score": scores, "Answer": answers, "Answer Score": answer_scores})
    df = df.sort_values(by=['Answer Score'], ascending=False)
    display(df)


### Questions

In [ ]:
best_answers("¿Se puede considerar 'gepetear' un neologismo?", 3)

,Document,Document Score,Answer,Answer Score
0,"El verbo gepetear es un neologismo válido que se emplea en ocasiones con el sentido de utilizar la aplicación ChatGPT. Debido al incremento del uso de esta herramienta de inteligencia artificial, está empezando a emplearse, especialmente en redes sociales, dicha voz para referirse al hecho de interactuar con este sistema para la resolución de dudas o por entretenimiento. Es un derivado formado a partir del deletreo en español de la sigla (ge-pe-te, no yi-pi-ti) que compone el nombre propio (GPT, de generative pre-trained transformer), al que se le ha añadido el sufijo productivo -⁠ear (como instagramear, de Instagram): gepetear. Asimismo, y siguiendo la misma pauta, para aludir al usuario, es posible crear gepetero o gepetista, al igual que ocurre con cenetista (de la sigla CNT), recogido en el Diccionario de la lengua española. Por tanto, pueden considerarse válidas frases como «La nueva forma de buscar y preguntar por información es gepetear» y «Estas son las búsquedas más frecuentes de los gepeteros/gepetistas».",18.126450,"valido que se emplea en ocasiones con el sentido de utilizar la aplicacion chatgpt . debido al incremento del uso de esta herramienta de inteligencia artificial , esta empezando a emplearse , especialmente en redes sociales , dicha voz para referirse al hecho de interactuar con este sistema para la resolucion de dudas o por entretenimiento . es un derivado formado a partir del deletreo en espanol de la sigla ( ge - pe - te , no yi - pi - ti ) que compone el nombre propio ( gpt , de generative pre - trained transformer ) , al que se le ha anadido el sufijo productivo - ear ( como instagramear , de instagram ) : gepetear . asimismo , y siguiendo la misma pauta , para aludir al usuario , es posible crear gepetero o gepetista , al igual que ocurre con cenetista ( de la sigla cnt ) , recogido en el diccionario de la lengua espanola . por tanto , pueden considerarse validas frases como « la nueva forma de buscar y preguntar por informacion es gepetear »",2.705511
1,"La locución adjetiva sin igual puede escribirse en una sola palabra. La nueva Ortografía de la lengua española ya recoge la escritura de la locución sin igual en una sola palabra: sinigual. Así, se podrían considerar igualmente adecuados los siguientes ejemplos: «Andalucía acumula parajes de sinigual belleza», «El Real Madrid, un equipo sin igual». Ver sin-, prefijo.",14.140447,igualmente adecuados los siguientes ejemplos,1.269745
2,"Se recomienda evitar el anglicismo decoupling en las informaciones económicas y usar en su lugar términos como desacople. Es fácil encontrar en medios de comunicación y en declaraciones de analistas financieros frases como «¿Es factible considerar un decoupling entre la demanda norteamericana y la de los países emergentes?», «En los últimos días se ha hablado mucho de este decoupling que para el FMI responde a una sobreinversión» o «En los meses previos a la caída de Lehman Brothers y la profunda crisis posterior, los analistas intentaban ver un potencial decoupling». El término inglés decoupling se refiere a la desconexión o falta de sincronía en la fase del ciclo económico o financiero en que se encuentran dos países o áreas económicas, lo que puede justificar que sus niveles de tipos o sus políticas económicas sean distintas. Aunque este anglicismo admite en español varias traducciones, como desacople, desajuste, desconexión o desacoplamiento, por un criterio de uniformidad y por su mayor concisión, se recomienda la primera alternativa. Así, en los ejemplos anteriores habría sido preferible escribir «¿Es factible considerar un desacople entre la demanda norteamericana y la de los países emergentes?», «En los últimos días se ha hablado mucho de este desacople que para el FMI responde a una sobreinversión» y «En los meses previos a la caída de Lehman Brothers y la profunda crisis posterior, los analistas intentaban ver un potencial desajuste».",13.895168,« ¿ es factible 

In [ ]:
best_answers("¿Cuáles son los equivalentes de 'password'?", 3)

,Document,Document Score,Answer,Answer Score
0,"Contraseña y clave/código de acceso son algunos de los equivalentes del extranjerismo inglés password. Sin embargo, es frecuente ver frases en la prensa como «Antes de restaurar el password, mejor utiliza estos sistemas para saber cuál es», «Es recomendable cambiar tu password con regularidad para evitar que alguien la descifre» o «Para conseguir un password relativamente seguro hay que alcanzar los diez caracteres». Con el sentido de ‘secuencia de caracteres que permite el acceso a un sistema’, con el que se emplea el anglicismo password, existen opciones en español como contraseña, clave/código de acceso, código de seguridad o clave personal, tal como señala el Diccionario panhispánico de dudas. Por tanto, es innecesario el uso del extranjerismo y se recomienda optar por alguna de las alternativas anteriores. Así, en las oraciones iniciales, lo idóneo habría sido escribir «Antes de restaurar la contraseña, mejor utiliza estos sistemas para saber cuál es», «Es recomendable cambiar tu clave de acceso con regularidad para evitar que alguien la descifre» y «Para conseguir una contraseña relativamente segura hay que alcanzar los diez caracteres».",17.768051,contrasena y clave / codigo de acceso,8.847131
1,"El uso del verbo descambiar con el sentido de ‘devolver una compra’ es adecuado. Pasadas las fiestas navideñas y con la llegada de las rebajas, muchas personas acuden a los comercios a devolver productos que compraron o les regalaron. Para referirse a ello se emplea a menudo el verbo descambiar, rechazado por quienes opinan que en este caso habría que utilizar cambiar: «Si quieres descambiar el producto que has comprado, tan solo tendrás que introducirlo de vuelta en la bolsa con esa hoja de devolución rellena», «Lo único que no se puede descambiar son los electrodomésticos» o «¿Cuál es el plazo para descambiar un artículo?». Sin embargo, como indica el Diccionario panhispánico de dudas, este sentido de descambiar, equivalente a ‘devolver una compra’, no se considera incorrecto porque no se trata más que de deshacer un cambio previo realizado en el momento de la adquisición y es un uso muy extendido en la lengua coloquial en España. Por tanto, los ejemplos anteriores pueden considerarse apropiados. En amplias áreas del español americano se utiliza también descambiar, pero con otro significado recogido tanto en el Diccionario de la lengua española como en el Diccionario de americanismos: ‘Convertir billetes o monedas grandes en dinero menudo equivalente o a la inversa’.",16.576988,« lo unico que no se puede descambiar son los electrodomesticos,3.849703
2,"Cuál se escribe con tilde cuando es palabra interrogativa o exclamativa. El término cual, sin tilde, puede ser un pronombre relativo, como en la frase «Ayer hablé con Isabel, la cual me dio recuerdos para todos», o un adverbio de modo que equivale a como: «Alberto ha resurgido, cual ave Fénix, de los altibajos de estos últimos meses». Se escribe con tilde, cuál, cuando es interrogativo o exclamativo: «¿Cuál es tu color favorito?», «Dime cuál te gusta más»; «Ese coche es el nuestro, ¡cuál va a ser!». Son inapropiadas las formas cuála, cuálo y sus plurales cuálas y cuálos. Véase también tilde diacrítica.",16.083534,"« alberto ha resurgido , cual ave fenix , de los altibajos de estos ultimos meses »",-0.871903


In [ ]:
best_answers("¿Cómo se escribe 'reduflacción' o 'reduflación?'", 3)

,Document,Document Score,Answer,Answer Score
0,"El término reduflación se escribe con una ce, no reduflacción, y no necesita comillas ni cursiva. Sin embargo, a veces se encuentran en los medios frases como «Hasta el monstruo de las galletas se ha dado cuenta de la reduflacción», «La ‘reduflación’ llega a los productos de chocolate con la subida del precio del cacao» o «Menos por más: ¿cómo la ‘reduflación’ impacta en alimentos y productos de limpieza?». La palabra reduflación, que el Diccionario del español actual, de Andrés, Seco y Ramos, define como ‘práctica que consiste en disminuir el tamaño o la cantidad de un producto manteniendo o elevando ligeramente el precio para disimular la inflación’, se escribe con una ce, no reduflacción. Esta voz está formada por los sustantivos reducción e inflación (no inflacción), siguiendo el modelo del anglicismo shrinkflation, a partir de shrink (‘reducción’) e inflation (‘inflación’). Por otro lado, puede utilizarse sin comillas ni cursiva al estar lo suficientemente asentada. Así pues, en los ejemplos iniciales, lo adecuado habría sido escribir: «Hasta el monstruo de las galletas se ha dado cuenta de la reduflación», «La reduflación llega a los productos de chocolate con la subida del precio del cacao» y «Menos por más: ¿cómo la reduflación impacta en alimentos y productos de limpieza?».",39.899457,el termino reduflacion,2.471524
1,"El término reduflación se escribe con una ce, no reduflacción, y no necesita comillas ni cursiva. Sin embargo, a veces se encuentran en los medios frases como «Hasta el monstruo de las galletas se ha dado cuenta de la reduflacción», «La ‘reduflación’ llega a los productos de chocolate con la subida del precio del cacao» o «Menos por más: ¿cómo la ‘reduflación’ impacta en alimentos y productos de limpieza?». La palabra reduflación, que el Diccionario del español actual, de Andrés, Seco y Ramos, define como ‘práctica que consiste en disminuir el tamaño o la cantidad de un producto manteniendo o elevando ligeramente el precio para disimular la inflación’, se escribe con una ce, no reduflacción. Esta voz está formada por los sustantivos reducción e inflación (no inflacción), siguiendo el modelo del anglicismo shrinkflation, a partir de shrink (‘reducción’) e inflation (‘inflación’). Por otro lado, puede utilizarse sin comillas ni cursiva al estar lo suficientemente asentada. Así pues, en los ejemplos iniciales, lo adecuado habría sido escribir: «Hasta el monstruo de las galletas se ha dado cuenta de la reduflación», «La reduflación llega a los productos de chocolate con la subida del precio del cacao» y «Menos por más: ¿cómo la reduflación impacta en alimentos y productos de limpieza?».",39.899457,el termino reduflacion,2.471524
2,"El término reduflación se escribe con una ce, no reduflacción, y no necesita comillas ni cursiva. Sin embargo, a veces se encuentran en los medios frases como «Hasta el monstruo de las galletas se ha dado cuenta de la reduflacción», «La ‘reduflación’ llega a los productos de chocolate con la subida del precio del cacao» o «Menos por más: ¿cómo la ‘reduflación’ impacta en alimentos y productos de limpieza?». La palabra reduflación, que el Diccionario del español actual, de Andrés, Seco y Ramos, define como ‘práctica que consiste en disminuir el tamaño o la cantidad de un producto manteniendo o elevando ligeramente el precio para disimular la inflación’, se escribe con una ce, no reduflacción. Esta voz está formada por los sustantivos reducción e inflación (no inflacción), siguiendo el modelo del anglicismo shrinkflation, a partir de shrink (‘reducción’) e inflation (‘inflación’). Por otro lado, puede utilizarse sin comillas ni cursiva al estar lo suficientemente asentada. Así pues, en los ejemplos iniciales, lo adecuado habría sido escribir: «Hasta el monstruo de las galletas se ha dado cuenta de la reduflación», «La reduflación llega a los productos de chocolate con la subida del precio del cacao» y «Menos por más: ¿cómo la reduflac

In [ ]:
best_answers("¿Cómo se acentúa 'adecua' o 'adecúa'?", 3)

,Document,Document Score,Answer,Answer Score
0,"El verbo adecuar se puede acentuar como averiguar (adecua) o como actuar (adecúa). En la lengua general y en los medios de comunicación es frecuente la vacilación a la hora de acentuar el verbo adecuar. Así, podemos encontrar frases como «El sistema sanitario va al fracaso si no se adecua al enfermo crónico», «Los secretarios judiciales exigen que se adecúe el salario al trabajo que realizan» o «Las medidas del cuerpo perfecto son utópicas y no se adecuan a la realidad». Según señala el Diccionario panhispánico de dudas, tanto la forma con tilde como sin ella son correctas, ya que, aunque la acentuación que sigue el modelo de averiguar (adecua) es considerada más culta, también se acepta la que sigue el modelo de actuar (adecúa). Por lo tanto, todos los ejemplos anteriores son correctos. No obstante, si el verbo adecuar aparece más de una vez en un mismo texto, es importante optar por una única acentuación para así mantener la uniformidad y coherencia interna. Estas reglas son aplicables a otros verbos, como licuar o evacuar.",15.301104,el verbo adecuar se puede acentuar como averiguar ( adecua ) o como actuar ( adecua ),4.374103
1,"El verbo agriar(se) se puede acentuar como enviar (se agría) o como anunciar (se agria). Los medios de comunicación registran vacilaciones a la hora de acentuar el verbo agriar(se), ‘ poner(se) agrio’: «La campaña electoral griega se agria», «La polémica del vino se agría». Sin embargo, estas vacilaciones están justificadas, ya que, tal y como se recoge en el Diccionario panhispánico de dudas, ambas acentuaciones son apropiadas. Aunque agriar(se) se acentúa mayoritariamente como enviar (se agría), también se considera válida la acentuación como anunciar (se agria).",10.772767,"el verbo agriar ( se ) se puede acentuar como enviar ( se agria ) o como anunciar ( se agria ) . los medios de comunicacion registran vacilaciones a la hora de acentuar el verbo agriar ( se ) , ‘ poner ( se ) agrio ’ : « la campana electoral griega se agria » , « la polemica del vino se agria » . sin embargo , estas vacilaciones estan justificadas , ya que , tal y como se recoge en el diccionario panhispanico de dudas , ambas acentuaciones son apropiadas . aunque agriar ( se ) se acentua mayoritariamente como enviar ( se agria )",-0.363828
2,"En español, la palabra tremor es aguda, es decir, que en su pronunciación se acentúa la última sílaba. En las noticias de la radio y la televisión se emplea a menudo la palabra tremor, pero pronunciada como si fuese una voz inglesa o latina: trémor, con el acento en la primera sílaba En sismología, tremor alude a un terremoto característico de los volcanes que refleja modificaciones en su estado interno; también se emplea en medicina como sinónimo de temblor. Se trata, en español, de una palabra aguda, por lo que no está justificado pronunciarla como llana: trémor; solo se pronunciaría así si se escribiese con tilde en la e.",10.207119,en su pronunciacion se acentua la ultima silaba,-1.883731


In [ ]:
best_answers("¿Cuál es la diferencia entre 'sino' y 'si no'?", 3)

,Document,Document Score,Answer,Answer Score
1,"Los términos rúter, enrutador, direccionador o encaminador son alternativas preferibles a l anglicismo router. En los medios de comunicación, sobre todo en los ámbitos de la tecnología y la informática, aparece con frecuencia la voz inglesa: «Un router con hasta 6,6 Gbps», «El hacker pudo entrar en millones de sistemas de todo el mundo utilizando un sencillo equipo conectado a un router » o «¿Cuál es la diferencia entre un router y un módem?». Sin embargo, puesto que este aparato se ha convertido en algo presente e imprescindible en muchos hogares y oficinas, el uso de este vocablo se ha normalizado en la lengua general — como ya señalaba la Fundéu en 2006 — y, por tanto, resulta adecuado adaptarla al español con la grafía rúter (plural rúteres), que reproduce la pronunciación habitual en español del anglicismo y que el Diccionario registra como ‘dispositivo que distribuye el flujo de paquetes de información entre redes de la manera más eficaz’ y ‘programa informático que realiza las mismas funciones que un rúter’. Como alternativas de router, también son apropiados los sustantivos direccionador, encaminador o enrutador, este último recogido asimismo en el Diccionario. Por tanto, en los ejemplos anteriores habría sido preferible escribir «Un rúter con hasta 6,6 Gbps», «El hacker pudo entrar en millones de sistemas de todo el mundo utilizando un sencillo equipo conectado a un enrutador» y «¿Cuál es la diferencia entre un encaminador y un módem?». Se recuerda que, si se desea utilizar el término inglés, es preciso escribirlo entrecomillado o en cursiva.",22.536168,un router y un modem,4.293673
2,"Los términos rúter, enrutador, direccionador o encaminador son alternativas preferibles a l anglicismo router. En los medios de comunicación, sobre todo en los ámbitos de la tecnología y la informática, aparece con frecuencia la voz inglesa: «Un router con hasta 6,6 Gbps», «El hacker pudo entrar en millones de sistemas de todo el mundo utilizando un sencillo equipo conectado a un router » o «¿Cuál es la diferencia entre un router y un módem?». Sin embargo, puesto que este aparato se ha convertido en algo presente e imprescindible en muchos hogares y oficinas, el uso de este vocablo se ha normalizado en la lengua general — como ya señalaba la Fundéu en 2006 — y, por tanto, resulta adecuado adaptarla al español con la grafía rúter (plural rúteres), que reproduce la pronunciación habitual en español del anglicismo y que el Diccionario registra como ‘dispositivo que distribuye el flujo de paquetes de información entre redes de la manera más eficaz’ y ‘programa informático que realiza las mismas funciones que un rúter’. Como alternativas de router, también son apropiados los sustantivos direccionador, encaminador o enrutador, este último recogido asimismo en el Diccionario. Por tanto, en los ejemplos anteriores habría sido preferible escribir «Un rúter con hasta 6,6 Gbps», «El hacker pudo entrar en millones de sistemas de todo el mundo utilizando un sencillo equipo conectado a un enrutador» y «¿Cuál es la diferencia entre un encaminador y un módem?». Se recuerda que, si se desea utilizar el término inglés, es preciso escribirlo entrecomillado o en cursiva.",22.536168,un router y un modem,4.293673
0,"Para identificar dos realidades, la construcción adecuada es uno y otro, no un y otro. Sin embargo, es frecuente que los medios de comunicación empleen frases como «Los parones existentes entre un y otro rodaje provocan que algunas personas lleguen incluso a la ruina», «Cuáles son esas diferencias que se notan cuando se utilizan un y otro dispositivo» o «Tras una primera vuelta donde giraron a la par un y otro, Josito Di Palma se sumó a la pelea por la tercera posición». Según se explica en la Nueva gramática de la lengua española, cuando se coordinan dos cuantificadores (uno, alguno, ninguno…), estos toman su forma apocopada (un, algún, ningún) si el primero precede a un sustantivo (un día y otro, 

In [ ]:
best_answers("¿Cuál es la forma adecuada para abreviar 'Reino Unido?'", 3)

,Document,Document Score,Answer,Answer Score
0,"La forma más adecuada de abreviar el nombre del Reino Unido es R. U. y no UK, que está tomada del inglés. En las noticias sobre este país, no es raro ver que se emplea la forma inglesa: «Zelenski pide en UK el envío de cazas», «Los viajeros procedentes de UK tendrán que solicitar un permiso en su país» o «Sunak promete bajar la inflación en UK a la mitad». UK es la sigla del nombre inglés del país, United Kingdom, pero en español su denominación asentada es Reino Unido, de la que se deriva la abreviatura R. U., escrita con puntos y con un espacio. Tal como se indica en la cuenta de Twitter de la RAE, esta es la grafía habitualmente empleada y la aconsejable, de conformidad con el hecho de que se suele leer desarrollada y no deletreada, aunque señala que la sigla RU está correctamente formada. En cambio, sí es ortográficamente inapropiado dejar los puntos y omitir el espacio: R.U. Aunque en los titulares y los rótulos la abreviatura resulta útil, conviene sopesar si, fuera de los contextos donde es necesario ahorrar espacio, no sería preferible dar el nombre al completo. En cualquier caso, se recuerda que al mencionar este país lo usual es anteponer el artículo: el Reino Unido, el R. U. Por ello, en los ejemplos anteriores se podría haber escrito, por ejemplo, «Zelenski pide en el R. U. el envío de cazas», «Los viajeros procedentes del Reino Unido tendrán que solicitar un permiso en su país» y «Sunak promete bajar la inflación en el R. U. a la mitad».",29.926000,"r . u . y no uk , que esta tomada del ingles",11.561605
1,"La forma más adecuada de abreviar el nombre del Reino Unido es R. U. y no UK, que está tomada del inglés. En las noticias sobre este país, no es raro ver que se emplea la forma inglesa: «Zelenski pide en UK el envío de cazas», «Los viajeros procedentes de UK tendrán que solicitar un permiso en su país» o «Sunak promete bajar la inflación en UK a la mitad». UK es la sigla del nombre inglés del país, United Kingdom, pero en español su denominación asentada es Reino Unido, de la que se deriva la abreviatura R. U., escrita con puntos y con un espacio. Tal como se indica en la cuenta de Twitter de la RAE, esta es la grafía habitualmente empleada y la aconsejable, de conformidad con el hecho de que se suele leer desarrollada y no deletreada, aunque señala que la sigla RU está correctamente formada. En cambio, sí es ortográficamente inapropiado dejar los puntos y omitir el espacio: R.U. Aunque en los titulares y los rótulos la abreviatura resulta útil, conviene sopesar si, fuera de los contextos donde es necesario ahorrar espacio, no sería preferible dar el nombre al completo. En cualquier caso, se recuerda que al mencionar este país lo usual es anteponer el artículo: el Reino Unido, el R. U. Por ello, en los ejemplos anteriores se podría haber escrito, por ejemplo, «Zelenski pide en el R. U. el envío de cazas», «Los viajeros procedentes del Reino Unido tendrán que solicitar un permiso en su país» y «Sunak promete bajar la inflación en el R. U. a la mitad».",29.926000,"r . u . y no uk , que esta tomada del ingles",11.561605
2,"Con motivo de la celebración del referéndum sobre la posible salida del Reino Unido de la Unión europea, se ofrecen algunas claves para una adecuada redacción de las noticias relacionadas con este acontecimiento. El término brexit, con el que suele aludirse en los medios a una hipotética salida del Reino Unido de la Unión Europea, se escribe en cursiva por tratarse de un extranjerismo y con inicial minúscula por ser un nombre común. Lo mismo es aplicable al término bremain, formado con el mismo paradigma a partir de las palabras Britain y remain (‘quedarse’). Las formas referendo y referéndum (con tilde) son adecuadas para referirse al proceso por el que se someten al voto popular leyes o decisiones políticas. Sus plurales respectivos son referendos y referéndums, no referenda. Inglaterra, el Reino Unido y Gran Bretaña no son lo mismo. Inglaterra, 

In [ ]:
best_answers("¿La palabra 'solo' se escribe con tilde cuando es un adverbio?", 3)

,Document,Document Score,Answer,Answer Score
1,"Cuál se escribe con tilde cuando es palabra interrogativa o exclamativa. El término cual, sin tilde, puede ser un pronombre relativo, como en la frase «Ayer hablé con Isabel, la cual me dio recuerdos para todos», o un adverbio de modo que equivale a como: «Alberto ha resurgido, cual ave Fénix, de los altibajos de estos últimos meses». Se escribe con tilde, cuál, cuando es interrogativo o exclamativo: «¿Cuál es tu color favorito?», «Dime cuál te gusta más»; «Ese coche es el nuestro, ¡cuál va a ser!». Son inapropiadas las formas cuála, cuálo y sus plurales cuálas y cuálos. Véase también tilde diacrítica.",25.650462,cual,2.467400
2,"Cuál se escribe con tilde cuando es palabra interrogativa o exclamativa. El término cual, sin tilde, puede ser un pronombre relativo, como en la frase «Ayer hablé con Isabel, la cual me dio recuerdos para todos», o un adverbio de modo que equivale a como: «Alberto ha resurgido, cual ave Fénix, de los altibajos de estos últimos meses». Se escribe con tilde, cuál, cuando es interrogativo o exclamativo: «¿Cuál es tu color favorito?», «Dime cuál te gusta más»; «Ese coche es el nuestro, ¡cuál va a ser!». Son inapropiadas las formas cuála, cuálo y sus plurales cuálas y cuálos. Véase también tilde diacrítica.",25.650462,cual,2.467400
0,"Dónde se escribe con tilde cuando es palabra interrogativa o exclamativa. Se escribe donde, sin tilde, cuando podemos equiparar la función que tiene la oración que encabeza con un complemento de lugar («Yoli trabaja cerca de donde vive Paco»); pero se escribe con tilde en las oraciones interrogativas en las que se pregunta por el lugar («¿Dónde trabaja Yoli?» o «No estoy segura de dónde trabaja Yoli») y en las exclamativas («¡Dónde hemos ido a parar!»). El hecho de que esta palabra vaya en una oración interrogativa o exclamativa no significa que sea exclamativa o interrogativa; por tanto, no debe llevar tilde: «¿Dónde trabajas? ¿Donde me dijiste?».",25.823261,donde se escribe con tilde cuando es palabra interrogativa o exclamativa,2.113616


In [ ]:
best_answers("¿Cómo se escriben los títulos de las obras de creación?", 3)

,Document,Document Score,Answer,Answer Score
0,"Los títulos de las obras de creación se escriben en cursiva y con inicial mayúscula solo en la primera palabra y en los nombres propios. En los medios pueden encontrarse ejemplos como los siguientes: «Ha protagonizado algunas de las mejores películas españolas, como Tengo Ganas de Ti », «La cantante acaba de interpretar La Creación de Haydn» o «Picasso tomó la decisión de pintar, otra vez, Las Meninas ». Tal como recuerda la Ortografía de la lengua española, los títulos de obras de creación, ya sean libros, películas, cuadros, piezas musicales, programas radiofónicos o televisivos…, se escriben en cursiva y en minúscula, salvo la inicial de la primera palabra y los nombres propios si incluyen alguno: Cien años de soledad, La decisión de Sophie, Las cuatro estaciones, El nacimiento de Venus… Se escriben también en cursiva, sin necesidad de ningún destacado adicional, los títulos originales de obras no escritas en español, aplicando la norma española en las mayúsculas (The scarlet letter o A man for all seasons), aunque puede respetarse la forma de escribirlos en la lengua correspondiente (The Scarlet Letter o A Man for All Seasons). Así, en los ejemplos anteriores, lo más adecuado habría sido escribir «Ha protagonizado algunas de las mejores películas españolas, como Tengo ganas de ti », «La cantante acaba de interpretar La creación de Haydn» y «Picasso tomó la decisión de pintar, otra vez, Las meninas ». Ver también ➤ Ortotipografía de títulos de publicaciones (Wikilengua)",25.476334,en cursiva y con inicial mayuscula solo en la primera palabra y en los nombres propios,12.643207
1,"Los títulos de las obras de creación se escriben en cursiva y con inicial mayúscula solo en la primera palabra y en los nombres propios. En los medios pueden encontrarse ejemplos como los siguientes: «Ha protagonizado algunas de las mejores películas españolas, como Tengo Ganas de Ti », «La cantante acaba de interpretar La Creación de Haydn» o «Picasso tomó la decisión de pintar, otra vez, Las Meninas ». Tal como recuerda la Ortografía de la lengua española, los títulos de obras de creación, ya sean libros, películas, cuadros, piezas musicales, programas radiofónicos o televisivos…, se escriben en cursiva y en minúscula, salvo la inicial de la primera palabra y los nombres propios si incluyen alguno: Cien años de soledad, La decisión de Sophie, Las cuatro estaciones, El nacimiento de Venus… Se escriben también en cursiva, sin necesidad de ningún destacado adicional, los títulos originales de obras no escritas en español, aplicando la norma española en las mayúsculas (The scarlet letter o A man for all seasons), aunque puede respetarse la forma de escribirlos en la lengua correspondiente (The Scarlet Letter o A Man for All Seasons). Así, en los ejemplos anteriores, lo más adecuado habría sido escribir «Ha protagonizado algunas de las mejores películas españolas, como Tengo ganas de ti », «La cantante acaba de interpretar La creación de Haydn» y «Picasso tomó la decisión de pintar, otra vez, Las meninas ». Ver también ➤ Ortotipografía de títulos de publicaciones (Wikilengua)",25.476334,en cursiva y con inicial mayuscula solo en la primera palabra y en los nombres propios,12.643207
2,"Los títulos de las canciones se escriben con mayúscula inicial en la primera palabra y en cursiva o entrecomillados. En los medios de comunicación es frecuente encontrar citados los títulos de algunas canciones, como en «Sacerdote sorprende en misa con su versión religiosa de Despacito», «La compositora de “despacito” subraya que con este tema “el mundo entero está cantando en español”» o «La parodia de Despacito: la odias, pero no te puedes resistir al tema». La ortografía académica señala que las piezas o los textos que pueden funcionar de manera autónoma, pero que forman parte de una obra mayor que, a su vez, tiene su propio título (como los poemas o los cuentos de un libro, las canciones de un álbum o los artículos de un

In [ ]:
best_answers("¿Se puede escribir coma entre sujeto y verbo de una oración?", 3)

,Document,Document Score,Answer,Answer Score
0,"Entre el sujeto y el verbo de una oración no se escribe coma por regla general. En los medios de comunicación pueden encontrarse frases como «Ciclones como David, San Zenón y George, impactaron en el sur de la isla», «La inmigración de nacionales venezolanos al país, ha aumentado a raíz de la crisis política y económica» o «Una nueva guerra comercial con la actividad económica seriamente tocada, sería devastadora para cualquier atisbo de recuperación». La Ortografía de la lengua española tacha de incorrecto el uso de la coma entre el sujeto y el verbo —conocida de forma coloquial como coma criminal o coma asesina —, aun en casos en que el sujeto está formado por varios elementos separados por comas o es muy largo. Esta longitud puede hacer necesaria una pausa fónica para respirar, pero no resulta adecuado reflejar por escrito dicha interrupción en el habla mediante una coma, ya que rompe la dependencia sintáctica y semántica entre dos grupos estrechamente relacionados. Así pues, en los ejemplos iniciales lo adecuado habría sido escribir «Ciclones como David, San Zenón y George impactaron en el sur de la isla», «La inmigración de nacionales venezolanos al país ha aumentado a raíz de la crisis política y económica» y «Una nueva guerra comercial con la actividad económica seriamente tocada sería devastadora para cualquier atisbo de recuperación». No obstante, la Academia recoge algunas excepciones a esta norma, entre las que cabe destacar las dos siguientes: — Se escribe coma entre el sujeto y el verbo si en medio se intercala un inciso entre dos comas: «Su pirámide poblacional, con un 50% de menores de 20 años, ha permitido que los sistemas de salud africanos, los más débiles del mundo, ganen tiempo». — Se escribe coma obligatoriamente si el sujeto termina en etcétera o su abreviatura etc.: «Jueces, fiscales, forenses, funcionarios, etc., deben informar de forma urgente a su personal del contagio por coronavirus».",36.871704,no se escribe coma por regla general,5.741227
1,"Cuando el verbo caber va seguido de un infinitivo, se conjuga en singular. Entre los significados del verbo caber está el de ‘ser posible’ o ‘ser natural’ y el sujeto, «lo que cabe», puede ser una frase con el verbo en infinitivo: «Cabe esperar buenas noticias» o «Cabe decir que el resultado ha sido magnífico», ejemplos en los que caber está correctamente en singular, de acuerdo con el Diccionario panhispánico de dudas. Sin embargo, cuando después del verbo en infinitivo hay nombres en plural, se puede caer en el error de emplear caber en plural, como en las frases «No cabían esperar análisis más profundos» o «En esa situación caben recordar los precedentes establecidos», donde lo apropiado habría sido escribir «No cabía esperar…» y «En esa situación cabe recordar…». Una forma de evitar este error es recordar que caber equivale a ‘ser posible’ o ‘ser natural’ y ver si en ese caso dichas formas podrían emplearse en plural. Puesto que no se diría «De este disco son posibles destacar dos canciones», tampoco sería apropiado «De este disco caben destacar dos canciones».",23.637582,"puede ser una frase con el verbo en infinitivo : « cabe esperar buenas noticias » o « cabe decir que el resultado ha sido magnifico » , ejemplos en los que caber esta correctamente en singular , de acuerdo con el diccionario panhispanico de dudas . sin embargo , cuando despues del verbo en infinitivo hay nombres en plural , se puede caer en el error de emplear caber en plural",-0.381721
2,"El verbo constatar puede construirse con un sujeto agente («Pedro constató…») o de forma impersonal («Se constata que…»), pero no con un sujeto inanimado («Los datos constatan…»). En los medios de comunicación es frecuente encontrar frases como «Los datos constatan la mejora del mercado laboral» o «Los resultados constataron la ausencia de diferencias en cuanto al peso, longitud o perímetro craneal de los neonatos». El verbo constatar requiere de un sujet

In [ ]:
best_answers("¿Cuál es la forma recomendada para referirse a los habitantes de Ucrania?", 3)

,Document,Document Score,Answer,Answer Score
2,"Askenazí es la grafía recomendada del término que se utiliza para referirse al colectivo judío oriundo de Europa central y orienta l. En los medios de comunicación es común encontrar frases como «Hungría es el país con mayor porcentaje de habitantes de origen judío ashkenazí » o «El gran rabino ashkenazí de Israel comenzó su visita oficial a la Argentina con una serie de encuentros y reuniones que realizó en la sede de la AMIA». El Diccionario de la lengua española da como forma preferible askenazí (plural askenazís o asquenazíes) aunque también se consideran válidas la variante llana askenazi y las que emplean la letra q (asquenazí y asquenazi), aunque, como señala el Diccionario panhispánico de dudas, estas últimas son menos frecuentes. La misma obra considera que las grafías eskenazi o esquenazi, que se emplean a veces en el Cono Sur y que son frecuentes como apellidos, son desaconsejables como nombre común para designar esta rama del pueblo judío. Por tanto, en los ejemplos anteriores hubiese sido más adecuado escribir «Hungría es el país con mayor porcentaje de habitantes de origen judío askenazí» y «El gran rabino askenazí de Israel comenzó su visita oficial a la Argentina con una serie de encuentros y reuniones que realizó en la sede de la AMIA».",24.36715,askenazi,11.699491
0,"El gentilicio recomendado de Bosnia es bosnio. Bosnia es una de las repúblicas independientes, que ya existían como federadas, que surgió tras la desmembración de la República de Yugoslavia. La grafía recomendada para referirse a los habitantes de la república independiente de Bosnia es bosnio. El Diccionario de la lengua española recoge también bosniaco y bosníaco, como formas poco usadas, que remiten a bosnio, preferida por ser la habitual.",27.36058,la grafia recomendada para referirse a los habitantes de la republica independiente de bosnia es bosnio,9.840715
1,"El gentilicio recomendado de Bosnia es bosnio. Bosnia es una de las repúblicas independientes, que ya existían como federadas, que surgió tras la desmembración de la República de Yugoslavia. La grafía recomendada para referirse a los habitantes de la república independiente de Bosnia es bosnio. El Diccionario de la lengua española recoge también bosniaco y bosníaco, como formas poco usadas, que remiten a bosnio, preferida por ser la habitual.",27.36058,la grafia recomendada para referirse a los habitantes de la republica independiente de bosnia es bosnio,9.840715


As we can observe, in most cases, the QA effectively retrieves the most pertinent recommendations for a given queary and extracts the segments from each document that is most likely to answer the question. Unlike ohter model we will examine later, this system does not generate answers, but it identifies and presents the most relevant information directly from the documents.

## Large Language Model

In this section, the aim is to observe how an LLM responds to 10 linguistic questions, which will be used as prompts, before having specific contextual information. In other words, its prior metalinguistic knowledge of the Spanish language will be observed.

In this case, the selected model has been Mistral 7B, which is a highly efficient and powerful language model with 7 billion parameters. It outperforms the leading open 13 billion parameter model, Llama 2, in all tested benchmarks. Additionally, it surpasses the best available 34 billion parameter model, Llama 1, in tasks involving reasoning, mathematics, and code generation. The documentarion is available here: [Mistral7B](https://arxiv.org/pdf/2310.06825).

There is also a fine-tuned version Mistral 7B - Instruct, which excels in following instructions and outperforms the Llama 2 13B - chat model in both human and automated evaluations. Specifically, the model that was used was the fine-tuned version [mistralai/Mistral-7B-Instruct-v0.1](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1) obtained from Hugging Face.

This model was chosen over other models worked in class such as Llama or Gemma because, after conducting several tests, it demonstrated the best knowledge of Spanish. Additionally, Spanish language does not have yet a reference language model of its own.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Now we create a fuction that generates a response based on a input (a linguistic query in our case) using the mistralai/Mistral-7B-Instruct-v0.1 model, and it then returns the cleaned response, removing the instruction or the query and the special tokens.

In [ ]:
def generate_response(content):
    messages = [{"role": "user", "content": content}]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)
    model.to(device)
    generated_ids = model.generate(model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    cleaned_response = clean_response(decoded[0])

    return cleaned_response

def clean_response(response):
    start = response.find('[/INST]') + len('[/INST]')
    end = response.find('</s>', start)
    return response[start:end].strip()

### Questions

Next, we generate responses for each of the linguistic queries in order to compare the results after implementing RAG.

In [ ]:
generate_response("¿Se puede considerar 'gepetear' un neologismo?")

'No, \'gepetear\' no se puede considerar un neologismo. Este término es muy común en España y se usa para referirse a la práctica de besar en forma de juego o de "divertido".\n\nAunque se ha usado esta palabra tradicional en esta manera, no ha sido considerada oficial por la Real Academia Española, ya que no lo incluye en su diccionario de la lengua española. Por el otro lado, si se hubieran usado más explícitamente otras formas de abrazos como "besar", "abrazarse", o "purses" en lugares específicos, podría ser una forma de crear un neologismo.\n\nEl neologismo es el uso de palabras creadas nuevamente o expresiones compuestas nunca antes vistas en una lengua de comunicación, y que pueden ser asociadas a un significado común a través del uso.'

In [ ]:
generate_response("¿Cuáles son los equivalentes 'password' en español?")

'En español no hay un equivalente exacto para la palabra "password", aunque se suele utilizar el término "contraseña" o "clave". Otros términos relacionados que se utilizan a veces son "código", "código PIN" o "código de acceso".'

In [ ]:
generate_response("¿Cómo se escribe 'reduflacción' o 'reduflación'?")

'La palabra «reduflacción» o «reduflación» se escribe igualmente, pero se pronuncia diferentemente.\n\n"Reduflacción" se pronuncia como «RÉDUFFLÁKSHON» (con la tónica en la primer sílaba). La palabra se refiere al proceso de refucar o encoger algo hasta un estado más compacto o pequeño.\n\n"Reduflación", por otro lado, se pronuncia como «RÉDUFFLASHON» (con la tónica en la segunda sílaba). La palabra se refiere al proceso y estado de estar inflado, reducido o en un estado más bajo de presión de aire.\n\nAsí que dependiendo de su significado, se puede escribir con tónica en la primera o segunda sílaba.'

In [ ]:
generate_response("¿Cómo se acentúa 'adecua' o 'adecúa'?")

'La palabra "adecua" se acenta "ae-dec\'-u-a". La manera correcta de acentuar esta palabra es agregar el acento en el primer y el último sílaba, como se muestra arriba. Hay varias maneras de colocar el acento en las sílabas, así que algunas personas la escribirán con el acento en la segunda sílaba, como "ae-de-CUA". Sin embargo, la forma más común y correcta es la que se muestra arriba.'

In [ ]:
generate_response("¿Cuál es la diferencia entre 'sino' y 'si no'?")

'La palabra \'sino\' en español significa \'on the contrary\' o \'nevertheless\'. Por ejemplo: "Voy al cine, sino estoy cansado" (I\'m going to the cinema, but I\'m tired).\n\nLa expresión \'si no\', también en español, significa \'if not\'. Por ejemplo: "Me voy al cine si no tengo más trabajo por hacer" (I\'m going to the cinema if I don\'t have any more work to do).\n\nEn resumen, \'sino\' se utiliza cuando se quiere contraponer una afirmativa y se quiere expresar que algo estará igual o mejor de lo contrario de lo que se pensaba mientras \'si no\' se utiliza cuando se quiere expresar la negación de una afirmativa.'

In [ ]:
generate_response("¿Cuál es la forma adecuada para abreviar 'Reino Unido'?")

'La abreviatura más común y aceptada para referirse al Reino Unido es "UK". Otra abreviatura menos común pero también aceptada es "GB", que sigue las iniciales del nombre oficial del país en inglés: \'Great Britain\'.'

In [ ]:
generate_response("¿La palabra 'solo' se escribe con tilde cuando es un adverbio?")

'No, la palabra \'solo\' se escribe con una sola tilde cuando se utiliza como un adverbio para significar "solamente" o "merecimente". Por ejemplo: "Él bailó solo" (He danced alone). En otro contexto, si \'solo\' se utiliza como un adjectivo para describir a una persona sola, no se utiliza ninguna tilde. Por ejemplo: "La mujer está sola en la habitación" (The woman is alone in the room).'

In [ ]:
generate_response("¿Cómo se escriben los títulos de las obras de creación?")

'Los títulos de las obras de creación varían según el tipo de trabajo y la tradición cultural. Aquí hay algunos ejemplos de cómo se escriben los títulos de diferentes tipos de obras:\n\n1. Libros: Los títulos de libros suele estar en mayorcas en letras mayúsculas y se escriben en el perímetro superior del página título, por ejemplo: "Los Cien Anos de Soledad" de Gabriel García Márquez o "El cielo es azul" de Esther Perelman.\n\n2. Poesía: Los títulos de poemas suele estar en letras mayúsculas, aunque algunos poetas prefieren letras minúsculas. Se escribe en línea o en dos líneas en caso de ser más largo que una página. Por ejemplo: "Do not Go Gentle into That Good Night" de Dylan Thomas o "La rosa blanca" de Edgar Allan Poe.\n\n3. Música: Los títulos de canciones suele estar en letras minúsculas y se escriben en letras blancas sobre negro en los libretos de ópera y cantinas. Por ejemplo: "O Fortuna" de Opera Carmen o "Twinkle Twinkle Little Star".\n\n4. Cine: Los títulos de películas s

In [ ]:
generate_response("¿Se puede escribir coma entre sujeto y verbo de una oración?")

'No se puede escribir coma entre sujeto y verbo de una oración en español, ya que el orden sintáctico es sujeto-verbo-objeto. Esto significa que el sujeto debe encontrarse directamente antes del verbo y el objeto después del verbo. Comas se utilizan en ciertos casos después de uno o varios elementos en dentro de una oración o después de una lista de palabras para separar la información y facilitar la lectura.'

In [ ]:
generate_response("¿Cuál es la forma recomendada para referirse a los habitantes de Ucrania?")

'Es recomendado referirse a los habitantes de Ucrania como ucranianos o ucranieños, dependiendo de su género. Si se refiere al pueblo en general, se utiliza el plural "ucranianos". Si se referiría a un individuo específico, se utiliza el singular "ucraniano" o "ucraniana" según su género.'

## LLM + RAG

Now we will implement Retrieval-Augmented Generation (RAG) which consists in improving the precision and reliability of generative models by incorporating factual information retrieved from external sources.

In our case, the process starts with a linguistic query from the user (1), followed by a search for the most relevant documents through the information retrieval system (2). Then, recommendations from FundéuRAE are retrieved (3) and integrated into the prompt as context (4). Finally, the mistralai/Mistral-7B-Instruct-v0.1 generates an enriched response (5) and returns it to the user.

![RAG Diagram](https://drive.google.com/uc?export=view&id=1SW9zp8E5PM-3xNLXacfTqRWWYwR6Fsi2)


The subsequent code generates a response by first retrieving pertinent documents to provide context to the LLM. The retrieved context is then combined with the question and ensures the response is in Spanish. After generating the response using the LLM, it cleans the output by removing the instruction and special tokens. The main objective is that the model's answers align with the recommendations of the FundéuRAE, which are in turn based on the rules of the Royal Spanish Academy.

In [ ]:
def generate_response_RAG(question):
    docs, scores = retrieve_documents(question, topk=3)
    context = " ".join(docs)
    spanish = "Responde las preguntas siempre en español."
    context_question = f"{spanish} {context} {question}"
    messages = [{"role": "user", "content": context_question}]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)
    model.to(device)
    generated_ids = model.generate(model_inputs, pad_token_id=tokenizer.eos_token_id, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)

    cleaned_response = clean_response(decoded[0])

    return cleaned_response

def clean_response(response):
    start = response.find('[/INST]') + len('[/INST]')
    end = response.find('</s>', start)
    return response[start:end].strip()

### Questions

Next, we generate responses for each linguistic query, this time using RAG to enrich the results. In the following section, we will compare the responses before and after applying RAG.

In [ ]:
question = "¿Se puede considerar 'gepetear' un neologismo?"
response = generate_response_RAG(question)
print(response)

¡Sí! Gepetear es un neologismo válido en ciertas circunstancias, como cuando se utiliza para referirse al uso de la aplicación ChatGPT en redes sociales o entretenimiento. Esta locución adjetiva sin igual es un derivado del deleteir y el productivo -ear y ha sido aceptada en los medios de comunicación y declaraciones de analistas financieros como una forma de describir la interactividad por ChatGPT. Como se explica en la respuesta anterior, el término «sinigual» se utiliza en las publicaciones académicas para referirse a esta locución y se puede considerar un neologismo bien establecido.


In [ ]:
question = "¿Cuáles son los equivalentes 'password' en español?"
response = generate_response_RAG(question)
print(response)

Hay varios equivalentes del extranjerismo inglés 'password' en español, como contraseña, clave/código de acceso, código de seguridad o clave personal. Hay que evitar el uso de este extranjerismo ya que es innecesario. El verbo decambiar con el sentido de 'devolver una compra' es apropiado.


In [ ]:
response = generate_response_RAG("¿Cómo es 'reduflacción' o 'reduflación'?")
print(response)

La palabra correcta es "reduflación". La palabra es formada por los sustantivos reducción e inflación (no inflacción) y sigue el modelo del anglicismo shrinkflation. Es importante escribirla con una ce y sin comillas ni cursiva. Hay veces que la palabra se puede encontrar en expresiones o frases como "reduflacción" o "las 'reduflaciones' llegarán a los productos de chocolate con el aumento del precio del cacao" o "¿qué impacto tiene la 'reduflación' sobre las alimentos y los productos de limpieza?" de manera que se escrive de esta manera para estar lo suficientemente asentada en el uso, aunque el estándar es utilizar la palabra "reduflación" sin comillas ni cursiva.


In [ ]:
response = generate_response_RAG("¿Cómo se acentúa 'adecua' o 'adecúa'?")
print(response)

El verbo "adecuar" se pronuncia, según el diccionario panhispánico de dudas, de dos maneras: con el acento o sin él. Aunque la forma que sigue el modelo de averiguar (adecua) se considera más culta, la que sigue el modelo de actuar (adecúa) también es aceptable, en gran medida. Todos los ejemplos proporcionados anteriormente son correctos.


In [ ]:
response = generate_response_RAG("¿Cuál es la diferencia entre 'sino' y 'si no'?")
print(response)

"Sino" y "si no" son dos palabras con significados similares, pero la diferencia reside en su uso y su construcción en una frase.

La frase "sino" se utiliza para negar una afirmación previa, y se construye anteponiendo "sino" al verbo negativo "no". Por ejemplo, "Voy al cine" y "No voy al cine, sino al cine" significan lo mismo, pero "sino" es lo que indica que no se vaya al cine, sino al cine.

Por otro lado, "si no" se utiliza también como forma de negar una afirmación previa, pero se construye anteponiendo "si" al verbo negativo "no". En esta caso, "si" funciona como condición en el conjunto de una frase, indicando que la afirmación se realiza solo si no se cumpla la condición. Por ejemplo, "Me gusta la comida mexicana" y "Me gusta la comida mexicana, si no me encanta el guacamole" significan lo mismo que "Voy al cine" y "No voy al cine, sino al cine", pero "si no" es lo que indica que la afirmación se realiza solo si no se disfruta del guacamole.

En suma, "sino" se utiliza para n

In [ ]:
response = generate_response_RAG("¿Cuál es la forma adecuada para abreviar 'Reino Unido'?")
print(response)

La forma adecuada para abreviar 'Reino Unido' es R.U. y no UK, que es la sigla del nombre inglés del país, United Kingdom, pero en español su denominación asentada es Reino Unido, de la que se deriva la abreviatura R.U., escrita con puntos y con un espacio, de conformidad con lo que se indica en la cuenta de Twitter de la Real Academia Española (RAE).


In [ ]:
response = generate_response_RAG("¿La palabra 'solo' se escribe con tilde cuando es un adverbio?")
print(response)

No, la palabra "solo" se escribe sin tilde cuando se usa como un adverbio. Por ejemplo, "Solo tengo un coche" o "Solo estoy en la casa".

Si "solo" se usa como un adjectivo, utilizando tilde, por ejemplo, en la frase "Es un hombre solo" o "Este libros son solo parcialmente de mi interés".

También, si "solo" se usa como un pronombre relativo o interrogativo, utilizando tilde, por ejemplo, "Solo le di una limón" o "¿Solo va a la clase ahora?".


In [ ]:
response = generate_response_RAG("¿Cómo se escriben los títulos de las obras de creación?")
print(response)

Los títulos de obras de creación se escriben en cursiva y con inicial mayúscula solo en la primera palabra y en los nombres propios.


In [ ]:
response = generate_response_RAG("¿Se puede escribir coma entre sujeto y verbo de una oración?")
print(response)

No se puede escribir coma entre sujeto y verbo de una oración. En general, entre el sujeto y el verbo no se escribe coma, ya que esto rompe la dependencia sintáctica y semántica entre dos grupos estrechamente relacionados. La excepción son las dos siguientes: — Se escribe coma entre sujeto y verbo si en medio se intercala un inciso entre dos comas: «Su pirámida poblacional, con un 50% de menores de 20 años, ha permitido que los sistemas de salud africanos, los más débiles del mundo, ganen tiempo». — Se escribe coma obligatoriamente si el sujeto termina en etcétera o su abreviatura etc.: «Jueces, fiscales, forenses, funcionarios, etc., deben informar de forma urgente a su personal del contagio por coronavirus».


In [ ]:
response = generate_response_RAG("¿Cuál es la forma recomendada para referirse a los habitantes de Ucrania?")
print(response)

La forma recomendada para referirse a los habitantes de Ucrania es ucraniano. Ucrania es una de las repúblicas independientes que surgieron tras la desmembración de la República de Yugoslavia. Ucrania tiene su propia cultura y idioma, el cual se ha vuelto más significativo desde la independencia del país. En España, el uso del término "ucraniano" es común en medios de comunicación, literatura y educación, por lo que es la principal forma preferida para referirse a los habitantes de la República de Ucrania. Otra forma menos común para referirse a los ucranianos es ucríano, de carácter llano o ucrínico, que son variantes del mismo idioma, aunque no está tan extendido en España como "ucraniano".


## Questions analysis

Here, the impact of RAG will be analyzed, comparing the model's responses to the 10 linguistic queries before and after implementing this technique.

### Table of results

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-8jvv{border-color:inherit;font-size:15px;text-align:left;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-fymr{border-color:inherit;font-weight:bold;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0pky"></th>
    <th class="tg-8jvv"><span style="font-weight:bold">COMPARISON OF ANSWERS WITH AND WITHOUT RAG</span></th>
    <th class="tg-0pky"></th>
    <th class="tg-0pky"></th>
    <th class="tg-0pky"></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0pky"></td>
    <td class="tg-8jvv"><span style="font-weight:bold">LLM (MISTRAL)</span></td>
    <td class="tg-0pky"><span style="font-weight:bold">LLM (MISTRAL) + RAG</span></td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 1</td>
    <td class="tg-0pky">No, 'gepetear' no se puede considerar un neologismo. Este término es muy común en España y se usa para referirse a la   práctica de besar en forma de juego o de "divertido".<br>   <br>Aunque se ha usado esta palabra tradicional en esta manera, no ha sido considerada oficial por la Real Academia Española, ya que no lo incluye en su diccionario de la lengua española. <br><br>Por el otro   lado, si se hubieran usado más explícitamente otras formas de abrazos como "besar", "abrazarse", o "purses" en lugares específicos, podría ser una forma de crear un neologismo.<br>   <br>El neologismo es el uso de palabras   creadas nuevamente o expresiones compuestas nunca antes vistas en una lengua   de comunicación, y que pueden ser asociadas a un significado común a través   del uso.   </td>
    <td class="tg-0pky">   <br>¡Sí! Gepetear es un neologismo válido en   ciertas circunstancias, como cuando se utiliza para referirse al uso de la   aplicación ChatGPT en redes sociales o entretenimiento. <br><br>Esta locución adjetiva sin igual es un derivado del deleteir y el productivo -ear y ha sido aceptada en los medios de comunicación y declaraciones de analistas financieros como una forma de describir la interactividad por ChatGPT. <br><br>Como se explica en la respuesta anterior, el término «sinigual» se utiliza en las publicaciones académicas para referirse a esta locución y se puede considerar un neologismo bien establecido.   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 2</td>
    <td class="tg-0pky">En español no hay un equivalente exacto para la palabra "password", aunque se suele utilizar el término "contraseña" o "clave". Otros términos relacionados que   se utilizan a veces son "código", "código PIN" o   "código de acceso".   <br></td>
    <td class="tg-0pky">&nbsp;&nbsp;&nbsp;<br>Hay varios equivalentes del&nbsp;&nbsp;&nbsp;extranjerismo inglés 'password' en español, como contraseña, clave/código de&nbsp;&nbsp;&nbsp;acceso, código de seguridad o clave personal. Hay que evitar el uso de este&nbsp;&nbsp;&nbsp;extranjerismo ya que es innecesario. El verbo decambiar con el sentido de&nbsp;&nbsp;&nbsp;'devolver una compra' es apropiado.&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 3</td>
    <td class="tg-0pky">La palabra «reduflacción» o «reduflación» se escribe igualmente, pero se pronuncia diferentemente.<br><br>"Reduflacción" se pronuncia como «RÉDUFFLÁKSHON» (con la tónica en la primer sílaba). La palabra se refiere al proceso de refucar o encoger algo hasta un estado más compacto o pequeño.<br>   <br> "Reduflación", por otro lado, se pronuncia como «RÉDUFFLASHON» (con la tónica en la segunda sílaba). La palabra se refiere al proceso y estado de estar inflado, reducido o en un estado más bajo de presión de aire.<br>   <br>Así que dependiendo de su significado,   se puede escribir con tónica en la primera o segunda sílaba.   </td>
    <td class="tg-0pky">   <br>La palabra correcta es "reduflación".   La palabra es formada por los sustantivos reducción e inflación (no   inflacción) y sigue el modelo del anglicismo shrinkflation. <br><br>Es importante   escribirla con una ce y sin comillas ni cursiva. Hay veces que la palabra se puede encontrar en expresiones o frases como "reduflacción" o   "las 'reduflaciones' llegarán a los productos de chocolate con el   aumento del precio del cacao"<br><br>o "¿qué impacto tiene la   'reduflación' sobre las alimentos y los productos de limpieza?" de manera   que se escrive de esta manera para estar lo suficientemente asentada en el   uso, aunque el estándar es utilizar la palabra "reduflación" sin   comillas ni cursiva.   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 4</td>
    <td class="tg-0pky">La palabra "adecua" se acenta   "ae-dec'-u-a". La manera correcta de acentuar esta palabra es   agregar el acento en el primer y el último sílaba, como se muestra arriba.   Hay varias maneras de colocar el acento en las sílabas, así que algunas   personas la escribirán con el acento en la segunda sílaba, como   "ae-de-CUA". <br><br>Sin embargo, la forma más común y correcta es la que   se muestra arriba.   <br></td>
    <td class="tg-0pky">El verbo "adecuar" se   pronuncia, según el diccionario panhispánico de dudas, de dos maneras: con el   acento o sin él. Aunque la forma que sigue el modelo de averiguar (adecua) se   considera más culta, la que sigue el modelo de actuar (adecúa) también es   aceptable, en gran medida. <br><br>Todos los ejemplos proporcionados anteriormente   son correctos.   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 5</td>
    <td class="tg-0pky">La palabra 'sino' en español significa   'on the contrary' o 'nevertheless'. Por ejemplo: "Voy al cine, sino   estoy cansado" (I'm going to the cinema, but I'm tired).<br>      <br><br>La expresión 'si no', también en   español, significa 'if not'. Por ejemplo: "Me voy al cine si no tengo   más trabajo por hacer" (I'm going to the cinema if I don't have any more   work to do).<br>      <br>En resumen, 'sino' se utiliza cuando se   quiere contraponer una afirmativa y se quiere expresar que algo estará igual   o mejor de lo contrario de lo que se pensaba mientras 'si no' se utiliza   cuando se quiere expresar la negación de una afirmativa.   </td>
    <td class="tg-0pky"> "Sino" y "si no" son   dos palabras con significados similares, pero la diferencia reside en su uso   y su construcción en una frase.<br>   <br>La frase "sino" se utiliza   para negar una afirmación previa, y se construye anteponiendo   "sino" al verbo negativo "no". Por ejemplo, "Voy al   cine" y "No voy al cine, sino al cine" significan lo mismo,   pero "sino" es lo que indica que no se vaya al cine, sino al cine.<br>   <br>Por otro lado, "si no" se   utiliza también como forma de negar una afirmación previa, pero se construye   anteponiendo "si" al verbo negativo "no". En esta caso,   "si" funciona como condición en el conjunto de una frase, indicando   que la afirmación se realiza solo si no se cumpla la condición. <br><br>Por ejemplo,   "Me gusta la comida mexicana" y "Me gusta la comida mexicana,   si no me encanta el guacamole" significan lo mismo que "Voy al   cine" y "No voy al cine, sino al cine", pero "si no"   es lo que indica que la afirmación se realiza solo si no se disfruta del   guacamole.<br>     <br>En suma, "sino" se utiliza   para negar una afirmación previa directamente, mientras que "si no"   se utiliza para negar una afirmación previa indirectamente, dentro del   contexto de una frase con condicación.   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 6</td>
    <td class="tg-0pky">&nbsp;&nbsp;&nbsp;<br>La abreviatura más común y aceptada para&nbsp;&nbsp;&nbsp;referirse al Reino Unido es "UK". Otra abreviatura menos común pero&nbsp;&nbsp;&nbsp;también aceptada es "GB", que sigue las iniciales del nombre&nbsp;&nbsp;&nbsp;oficial del país en inglés: 'Great Britain'.&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky">   <br>La forma adecuada para abreviar 'Reino   Unido' es R.U. y no UK, que es la sigla del nombre inglés del país, United   Kingdom.<br><br>En español su denominación asentada es Reino Unido, de la que   se deriva la abreviatura R.U., escrita con puntos y con un espacio, de   conformidad con lo que se indica en la cuenta de Twitter de la Real Academia   Española (RAE).   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 7</td>
    <td class="tg-0pky">No, la palabra 'solo' se escribe con una   sola tilde cuando se utiliza como un adverbio para significar   "solamente" o "merecimente". Por ejemplo: "Él bailó   solo" (He danced alone). <br><br>En otro contexto, si 'solo' se utiliza como un  adjectivo para describir a una persona sola, no se utiliza ninguna tilde. Por   ejemplo: "La mujer está sola en la habitación" (The woman is alone   in the room).   </td>
    <td class="tg-0pky">No, la palabra "solo" se   escribe sin tilde cuando se usa como un adverbio. Por ejemplo, "Solo   tengo un coche" o "Solo estoy en la casa".<br><br>Si "solo" se usa como un   adjectivo, utilizando tilde, por ejemplo, en la frase "Es un hombre   solo" o "Este libros son solo parcialmente de mi interés".<br>   <br>También, si "solo" se usa como   un pronombre relativo o interrogativo, utilizando tilde, por ejemplo,   "Solo le di una limón" o "¿Solo va a la clase ahora?".   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 8</td>
    <td class="tg-0pky"> Los títulos de las obras de creación   varían según el tipo de trabajo y la tradición cultural. Aquí hay algunos   ejemplos de cómo se escriben los títulos de diferentes tipos de obras:<br>   <br>1. Libros: Los títulos de libros suele   estar en mayorcas en letras mayúsculas y se escriben en el perímetro superior   del página título, por ejemplo: "Los Cien Anos de Soledad" de   Gabriel García Márquez o "El cielo es azul" de Esther Perelman.<br>   <br>2. Poesía: Los títulos de poemas suele   estar en letras mayúsculas, aunque algunos poetas prefieren letras   minúsculas. Se escribe en línea o en dos líneas en caso de ser más largo que   una página. Por ejemplo: "Do not Go Gentle into That Good Night" de   Dylan Thomas o "La rosa blanca" de Edgar Allan Poe.<br>   <br>3. Música: Los títulos de canciones   suele estar en letras minúsculas y se escriben en letras blancas sobre negro   en los libretos de ópera y cantinas. Por ejemplo: "O Fortuna" de   Opera Carmen o "Twinkle Twinkle Little Star".<br>   <br>4. Cine: Los títulos de películas suele   estar en mayúsculas y se escriben en letras blancas sobre negro en los   carteles de cine. Por ejemplo: "Titanic", "Inception" o   "Shawshank Redemption".<br>   <br>5. Arte: Los títulos de obras de arte   suele estar en mayúsculas y se escriben en letras blancas sobre negro en la   placa informativa de la pieza. Por ejemplo: "Guernica" de Pablo   Picasso o "Mona Lisa" de Leonardo da Vinci.<br><br>6. Juegos: Los títulos de videojuegos   suele estar en mayúsculas y se escriben en letras blancas sobre negro en la   pantalla de titulo de las consolas. Por ejemplo: "Super Mario   Bros.", "The Legend of Zelda" o "GTA V".   </td>
    <td class="tg-0pky">&nbsp;&nbsp;&nbsp;<br>Los títulos de obras de creación escriben en cursiva y con inicial mayúscula solo en la primera palabra y en los nombres propios.&nbsp;&nbsp;&nbsp;</td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 9</td>
    <td class="tg-0pky">No se puede escribir coma entre sujeto y   verbo de una oración en español, ya que el orden sintáctico es   sujeto-verbo-objeto. <br><br>Esto significa que el sujeto debe encontrarse   directamente antes del verbo y el objeto después del verbo. Comas se utilizan   en ciertos casos después de uno o varios elementos en dentro de una oración o   después de una lista de palabras para separar la información y facilitar la   lectura.   </td>
    <td class="tg-0pky">No se puede escribir coma entre sujeto y   verbo de una oración. En general, entre el sujeto y el verbo no se escribe   coma, ya que esto rompe la dependencia sintáctica y semántica entre dos   grupos estrechamente relacionados. <br><br>La excepción son las dos siguientes: — Se   escribe coma entre sujeto y verbo si en medio se intercala un inciso entre   dos comas: «Su pirámida poblacional, con un 50% de menores de 20 años, ha   permitido que los sistemas de salud africanos, los más débiles del mundo,   ganen tiempo». <br><br>— Se escribe coma obligatoriamente si el sujeto termina en   etcétera o su abreviatura etc.: «Jueces, fiscales, forenses, funcionarios,   etc., deben informar de forma urgente a su personal del contagio por   coronavirus».   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
  <tr>
    <td class="tg-fymr">Question 10</td>
    <td class="tg-0pky">Es recomendado referirse a los   habitantes de Ucrania como ucranianos o ucranieños, dependiendo de su género.   <br><br>Si se refiere al pueblo en general, se utiliza el plural   "ucranianos". Si se referiría a un individuo específico, se utiliza   el singular "ucraniano" o "ucraniana" según su género.   </td>
    <td class="tg-0pky">La forma recomendada para referirse a   los habitantes de Ucrania es ucraniano. Ucrania es una de las repúblicas   independientes que surgieron tras la desmembración de la República de   Yugoslavia. <br><br>Ucrania tiene su propia cultura y idioma, el cual se ha vuelto   más significativo desde la independencia del país. En España, el uso del   término "ucraniano" es común en medios de comunicación, literatura   y educación, por lo que es la principal forma preferida para referirse a los   habitantes de la República de Ucrania. <br><br>Otra forma menos común para referirse   a los ucranianos es ucríano, de carácter llano o ucrínico, que son variantes   del mismo idioma, aunque no está tan extendido en España como   "ucraniano".   </td>
    <td class="tg-0pky"></td>
    <td class="tg-0pky"></td>
  </tr>
</tbody>
</table>

### Qualitative analysis

In the comparison table of previous questions shown above, we can observe the responses provided by the Mistral-7B model and their comparison when RAG is performed, which, as explained in previous sections, involves generating text after searching a database of relevant documents for the given context. We will analyse if the responses improve, as well as if a lower degree of error is observed because it is based on retrieved information. The questions have been selected in a varied manner, and examples from several categories of FundéuRAE have been chosen.

*   In the first question, the model fails to recognize 'gepetear' as a neologism and incorrectly associates it with an 'acción de besar'(action of kissing). Additionally, it falsely claims that this term is widely accepted in Spain. Moreover, it provides an unnecessary definition of neologism. However, when compared to the RAG response, we note that, consistent with FundéuRAE's assertion, 'gepetear' is acknowledged as a neologism when contextualized within the usage of ChatGPT. Nevertheless, it inexplicably includes the unrelated term 'sinigual' at the end of its response.

*   In the second question, we observe that while the model without RAG recognized alternatives to the English term 'password,' it still contends that there are no exact equivalents. Subsequently, after gathering specific information from the dataset, it advocates for avoiding the use of loanwords as unnecessary, instead suggesting terms endorsed by the FundéuRAE. However, its response is imperfect as it adds irrelevant information once more at the end, introducing the nonexistent Spanish verb 'decambiar'.

*   The third question provides yet another clear demonstration of improved accuracy in responses after RAG. It delivers a concise answer, affirming that the word 'reduflación' should be written with a single 'c' and avoids any inaccuracies in the answer. Prior to this, the model erroneously mixed concepts, suggesting there might be two ways to pronounce the words, which is incorrect since there are no alternate meanings or changes in spelling.

*  The fourth question also showcases how, with the implementation of RAG, the model delivers coherent and precise responses. Both forms of accentuation, 'adecua' and 'adecúa,' are indeed correct. Before RAG, the model's precision falters, suggesting that the correct way to accentuate this word is to 'agregar el acento en el primer y último sílaba' (add the accent on the first and last syllable). Nevertheless, in Spanish, there are no words with double accentuation, so this assertion is incorrect.

* In the fifth question, we observe an improvement both in length and in the accuracy of the response after the implementation of RAG. In the first response, the examples are provided in English, which is not coherent when talking only in Spanish. However, the first answer is not inconsistent in the responses, especially in what it states in the 'en resumen' (summary) section. In the second response, it is more concise in the explanations and examples, although it also includes errors such as 'una frase con condicación' when it should refer to 'condicional'.

* The sixth question is a clear and visual example of the effect of RAG, as the Mistral model has been trained in English and returns the acronyms in this language in the first answer. After obtaining the information from FundéuRAE, we observe how it states that the appropriate form is 'R. U.', as the platform records.

*   The answers to the seventh question are more confusing. When the LLM responds for the first time, it adheres to the old norm of the RAE, where 'solo' was accented only when it was an adverb but not an adjective. Additionally, it returns examples in English.
However, in the second response, we notice a change: the LLM now aligns with the updated information that 'solo' is written without an accent when used as an adverb. Yet, it introduces confusion by stating that 'solo' should have an accent when acting as an adjective, contrary to the latest recommendations from FundéuRAE.

* The eighth question, before RAG, returned a lot of information divided by category, but mixed titles in both Spanish and English, where the rules are different. Additionally, it included irrelevant data such as 'se escriben en el perímetro superior del página título' (they are written on the upper perimeter of the page title) or 'se escriben en letras blancas sobre negro' (they are written in white letters on black). After RAG, it responds correctly and concisely, with the requested information extracted from the created dataset .

*  In the ninth question, the model answers accurately in both instances. However, after employing RAG, it presents improved examples and accurately identifies existing exceptions, following the structure established in the FundéuRAE dataset.

* Lastly, in question number ten, the model generates incorrect responses in both answer. 'Ucranieños', 'ucríano', and 'ucrínico' are not valid terms. The only accepted forms are 'ucraniano' or 'ucraniana', thus the model fails to deliver a precise answer.

## Conclusions

It can be stablished, as a conclusion, that the implementation of RAG significantly improves the responses to the linguistic queries, making them more concise and appropriate, while also providing new metalinguistic information to the model. Notable improvements include the evolution of the accuracy of responses, the inclusion of renewed content, and a reduction of irrelevant information. Additionally, there has been an improvement in the quality of examples, which are often obtained directly from FundéuRAE.

However, hallucinations are still present, and the output can also be highly variable and unpredictable, as we have obtained diverse responses using the same prompts. Persistent issues and misconceptions, such as the introduction of non-existent terms, remain a challenge. Finally, creating a chatbot, as proposed in the following section, with this technology would be very interesting to improve the user's information retrieval on Fundéu’s website.

# Chatbot

This small chatbot is an interactive Python program that allows the user to ask questions and receive responses using a function based on Retrieval-Augmented Generation (RAG). Although this is just a small test proporsal, it could be a very useful tool for FundéuRAE users, writers, journalists, or the general public who want quick solutions to language-related queries. With a larger and more thoroughly reviewed chatbot, users can get immediate answers without having to navigate through the website.

In [ ]:
def chatbot():
  print("¡Hola! ¿En qué puedo ayudarte?\n")

  while True:
    question = input()
    if question.lower() in ['exit', 'quit']:
            print("¡Hasta luego!")
            break

    response=generate_response_RAG(question)
    print(response, "\n")

In [ ]:
chatbot()

¡Hola! ¿En qué puedo ayudarte?

¿Qué es más adecuado 'confort' o 'comfort'?
El término más adecuado en español es 'confort'. Esta palabra se ha incorporated al léxico español a través del francés y se escribe sin una tilde, por acabar en más de una consonante. Otras equivalentes que se pueden usar en su lugar son 'benestar' o 'comodidad'. El uso de 'comfort' en español es posible, pero suele ser menos común y puede alternar con esos equivalentes. 

exit
¡Hasta luego!
